# Fred's Iowa Notebook
> A test notebook

Where did I get this notebook?

> twitter: https://twitter.com/313V/status/1225229621693730817?s=20

In [7]:
import requests as r
import pandas as pd
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

Get HTML

In [8]:
url = 'https://results.thecaucuses.org/'
resp = r.get(url)
soup = BeautifulSoup(resp.text);

Parse HTML

In [9]:
precinct_table = soup.find('div', {'class': 'precinct-table'})
thead = [li.text for li in soup.find('ul', {'class': 'thead'}).findAll('li')]
sub_head = [li.text for li in soup.find('ul', {'class': 'sub-head'}).findAll('li')]

In [10]:
candidates = [thead[i] for i in range(2,len(thead), 3)]
counties = precinct_table.findAll('div', attrs = {'class': 'precinct-rows'})
county = counties[0]

In [11]:
rows = []
for county in counties:
    county_name = county.find('div', {'class': 'precinct-county'}).find('div', {'class': 'wrap'}).text
    county_data = county.find('div', {'class': 'precinct-data'}).findAll('ul', lambda v: v != 'total-row')
    for row in county_data:
        row_data = [col.text for col in row.findAll('li')]
        precinct_name = row_data[0]
        for i in range(1,len(row_data), 3):
            first_exp, final_exp, sde = row_data[i:(i+3)]
            candidate = candidates[i//3]
            rows.append(dict(county=county_name, precinct=precinct_name, candidate=candidate, 
                             first_exp=int(first_exp), final_exp=int(final_exp), sde=float(sde)))

In [12]:
df = pd.DataFrame(rows).sort_values(['county', 'precinct', 'candidate'])

In [13]:
df.groupby(['candidate', 'county'])[['first_exp', 'final_exp', 'sde']].sum()

first_exp  final_exp     sde
candidate county                                              
Bennet    Adair                           0          0  0.0000
          Adams                           0          0  0.0000
          Allamakee                       1          0  0.0000
          Appanoose                       0          0  0.0000
          Audubon                         0          0  0.0000
          Benton                          0          0  0.0000
          Black Hawk                      2          0  0.0000
          Boone                           3          0  0.0000
          Bremer                          0          0  0.0000
          Buchanan                        0          0  0.0000
          Buena Vista                     0          0  0.0000
          Butler                          0          0  0.0000
          CD2 Satellite                   0          0  0.0000
          CD4 Satellite                   0          0  0.0000
          Calhoun                         0          0  0.0000
          Carroll                         0          0  0.0000
          Cass                            0          0  0.0000
          Cedar                           1          0  0.0000
          Cerro Gordo                     0          0  0.0000
          Cherokee                        0          0  0.0000
          Chickasaw                       0          0  0.0000
          Clarke                          0          0  0.0000
          Clay                            1          0  0.0000
          Clayton                         0          0  0.0000
          Clinton                         0          0  0.0000
          Crawford                        0          0  0.0000
          Dallas                          4          0  0.0000
          Davis                           0          0  0.0000
          Decatur                         1          0  0.0000
          Delaware                        0          0  0.0000
...                                     ...        ...     ...
Yang      O'Brien                         3          3  0.0727
          Osceola                         0          0  0.0000
          Out of State Satellite         23          7  0.0216
          Page                           16          7  0.0000
          Palo Alto                       4          0  0.0000
          Plymouth                       39         13  0.2250
          Pocahontas                     11          7  0.1200
          Polk                         1683         23  0.2798
          Pottawattamie                 172        114  1.9200
          Poweshiek                      91          0  0.0000
          Ringgold                        1          0  0.0000
          Sac                             5          3  0.0667
          Scott                         415         86  0.4133
          Shelby                          2          1  0.0000
          Sioux                          64         56  0.4375
          Story                         716        246  1.6600
          Tama                           15         14  0.2588
          Taylor                          7          6  0.2000
          Union                          26          7  0.1500
          Van Buren                       0          0  0.0000
          Wapello                        31         14  0.2700
          Warren                         87          3  0.0000
          Washington                     31          0  0.0000
          Wayne                           3          0  0.0000
          Webster                        36          4  0.0833
          Winnebago                      30         29  0.4500
          Winneshiek                    151         49  0.5120
          Woodbury                      159         40  0.6375
          Worth                          27         23  0.4000
          Wright                          5          0  0.0000

[1428 rows x 3 columns]

In [195]:
df.to_csv('iowa_data.csv', index=False)